### Using dataset from https://github.com/tomasonjo/blog-datasets/blob/main/movies/movies_small.csv

In [4]:
NEO4J_URI="neo4j+s://1f7a937f.databases.neo4j.io"
NEO4J_USERNAME="neo4j"
NEO4J_PASSWORD="aYCioexIrrl7g34zoj0XWnGTECT0a1NZkCrGDjXBI_c"

In [5]:
import os
os.environ["NEO4J_URI"] = NEO4J_URI
os.environ["NEO4J_USERNAME"] = NEO4J_USERNAME
os.environ["NEO4J_PASSWORD"] = NEO4J_PASSWORD

In [6]:
from langchain_neo4j import Neo4jGraph
graph = Neo4jGraph(url=NEO4J_URI, username=NEO4J_USERNAME, password=NEO4J_PASSWORD)
graph

In [46]:
movie_query = """

LOAD CSV WITH HEADERS FROM 'https://raw.githubusercontent.com/tomasonjo/blog-datasets/main/movies/movies_small.csv' AS row

MERGE (m:Movie {id: row.movieId})
SET m.released = date(row.released),
    m.title = row.title,
    m.imdbRating = toFloat(row.imdbRating)
FOREACH (director IN split(row.director, '|') | 
    MERGE (p:Person {name: trim(director)})
    MERGE (p)-[:DIRECTED]->(m))
FOREACH (actor IN split(row.actors, '|') | 
    MERGE (p:Person {name: trim(actor)})
    MERGE (p)-[:ACTED_IN]->(m))
FOREACH (genre IN split(row.genres, '|') | 
    MERGE (g:Genre {name: trim(genre)})
    MERGE (m)-[:IN_GENRE]->(g))

"""

In [ ]:
#WITH row WHERE row.movieId IS NOT NULL AND trim(row.movieId) <> ''

In [47]:
movie_query

"\n\nLOAD CSV WITH HEADERS FROM 'https://raw.githubusercontent.com/tomasonjo/blog-datasets/main/movies/movies_small.csv' AS row\n\nMERGE (m:Movie {id: row.movieId})\nSET m.released = date(row.released),\n    m.title = row.title,\n    m.imdbRating = toFloat(row.imdbRating)\nFOREACH (director IN split(row.director, '|') | \n    MERGE (p:Person {name: trim(director)})\n    MERGE (p)-[:DIRECTED]->(m))\nFOREACH (actor IN split(row.actors, '|') | \n    MERGE (p:Person {name: trim(actor)})\n    MERGE (p)-[:ACTED_IN]->(m))\nFOREACH (genre IN split(row.genres, '|') | \n    MERGE (g:Genre {name: trim(genre)})\n    MERGE (m)-[:IN_GENRE]->(g))\n\n"

In [48]:
graph.query(movie_query)

[]

In [49]:
graph.refresh_schema()
print(graph.schema)

Node properties:
CEO {name: STRING, POB: STRING, YOB: INTEGER}
Company {name: STRING}
entreprenur {name: STRING, POB: STRING, YOB: INTEGER}
Country {name: STRING}
Person {name: STRING, born: INTEGER}
Movie {title: STRING, released: INTEGER, id: STRING, imdbRating: FLOAT}
Genre {name: STRING}
Relationship properties:

The relationships:
(:Person)-[:ACTED_IN]->(:Movie)
(:Person)-[:DIRECTED]->(:Movie)
(:Movie)-[:IN_GENRE]->(:Genre)
